In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory, SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [3]:


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_chat_history(user_id):
    return SQLChatMessageHistory(
        table_name='customer',
        session_id=user_id,
        connection="sqlite:///sqlite.db",
    )

In [4]:
# 2. 질문 프롬프트
order_system_prompt = """
    당신은 아이스크림 토핑 주문 접수원입니다. 반드시 존대말을 사용하세요.
    우리 가게에 아이스크림 토핑 종류는 3개가 있습니다.
    당신은 고객을 기억하고 고객과의 대화를 기억합니다.
    고객이 과거 자신의 주문 내역을 요청하면, 대답해주세요. 
    
    고객이 회원가입을 희망하는 경우, '회원가입이 완료되었습니다.'라고 대답하세요.
    고객이 토핑 또는 메뉴의 종류를 물어볼 경우, '코코볼, 아몬드, 시리얼 3가지 토핑이 있습니다.'라고 대답하세요.
    
    
    만약 고객이 아이스크림 토핑 주문을 한 경우,
    받은 주문을 짧고 간결하게 확인하는 문장을 제공하세요. 
    주문이 접수되었다면 반드시 마지막 문장은 '주문되었습니다.'로 말하세요.
    다음은 주문과 답변 예시입니다. 아래와 같이 대답하세요.
    
    고객)
    아몬드랑 시리얼 주세요!!
    
    AI접수원)
    
    - 아몬드
    - 시리얼
    
    주문되었습니다.
    
    
    위 규칙을 통해 아래 고객의 주문에 대해 답변해주세요. 
    

    고객):
    {input}

    AI접수원):
    """

In [5]:
# # 프롬프트 정의
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
#         ),
#         # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
#         MessagesPlaceholder(variable_name="chat_history"),
#         ("human", "#Question:\n{input}"),  # 사용자 입력을 변수로 사용
#     ]
# )

prompt = ChatPromptTemplate.from_messages([
    ("system", order_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# llm 생성
llm = ChatOpenAI(model_name="gpt-4o")

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

In [6]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    )
]

In [7]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수를 설정합니다.
    input_messages_key="input",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history",  # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields,  # 대화 기록 조회시 참고할 파라미터를 설정합니다.
)

In [8]:

question = "안녕 반가워, 내 이름은 김현우야."


response = chain_with_history.invoke(
            {"input": question},
            config={"configurable": {"user_id": "user1"}}            # 같은 session_id 를 입력하면 이전 대화 스레드의 내용을 가져오기 때문에 이어서 대화가 가능!
        )

response

'안녕하세요, 김현우 고객님. 만나서 반갑습니다! 아이스크림 토핑 주문하시거나 궁금하신 점 있으시면 말씀해 주세요.'

In [9]:

question = "내 이름이 뭐라고?"


response = chain_with_history.invoke(
            {"input": question},
            config={"configurable": {"user_id": "user1"}}            # 같은 session_id 를 입력하면 이전 대화 스레드의 내용을 가져오기 때문에 이어서 대화가 가능!
        )

response

'김현우 고객님이십니다. 주문하시거나 궁금한 점 있으시면 언제든지 말씀해 주세요.'

In [ ]:

question = "코코볼 주세요"


response = chain_with_history.invoke(
            {"input": question},
            config={"configurable": {"user_id": "user1"}}            # 같은 session_id 를 입력하면 이전 대화 스레드의 내용을 가져오기 때문에 이어서 대화가 가능!
        )

response

## 만든 sqlite.db 보기

In [10]:
user_id = "user1"  # 대화 기록을 조회할 사용자 ID
chat_history = SQLChatMessageHistory(
    table_name="customer",  # 대화 기록이 저장된 테이블 이름
    session_id=user_id,     # 조회할 세션 ID
    connection="sqlite:///sqlite.db",  # SQLite 데이터베이스 연결
)

In [11]:
messages = chat_history.messages

messages

[HumanMessage(content='안녕 반가워, 내 이름은 김현우야.'),
 AIMessage(content='안녕하세요, 김현우 고객님. 만나서 반갑습니다! 아이스크림 토핑 주문하시거나 궁금하신 점 있으시면 말씀해 주세요.'),
 HumanMessage(content='내 이름이 뭐라고?'),
 AIMessage(content='김현우 고객님이십니다. 주문하시거나 궁금한 점 있으시면 언제든지 말씀해 주세요.')]